<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anytree

In [ ]:
!pip3 install convokit
!python3 -m spacy download en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.8 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216708 sha256=e83dfce4c649a740a1dee19240e020edc3b3df6b969d65f2c9af29619086bcd0
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1c33e0d5525b

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

In [ ]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [ ]:
import json

def count_comments_in_file(file_path):
    count = 0
    with open(file_path, 'r') as json_file:
        lines = json_file.readlines()
        for line in lines:
            data = json.loads(line)
            count += len(data.get('comments', []))
    return count

def total_comments_in_files(file_list_path):
    total_comments = 0
    with open(file_list_path, 'r') as file_list:
        for line in file_list:
            print(line)
            line = line.strip()
            if line:
                file_path, _ = line.split(',')
                total_comments += count_comments_in_file(f"{base_path}comments_{file_path}.json")
    return total_comments

# Replace 'file_list.txt' with the path to your text file containing the list of file names
total_comments = total_comments_in_files(f"{base_path}timestamps_seed2_num20_period10_date2022-10-1.txt_comments_progress.txt")
print(f'Total number of comments in the files: {total_comments}')

1665235800,1

1665097800,1

1664606400,1

1664775600,1

1664791200,1

1664845200,1

1664960400,1

1664638200,1

1665160800,1

1664823600,1

1664724000,1

1665093000,1

1664802000,1

1664841000,1

1664933400,1

1665264000,1

1664596200,1

1665165600,1

1665181200,1

1664805000,1

Total number of comments in the files: 665973


In [ ]:
comments_file = base_path + 'comments_1664638200.json'

In [ ]:
conversations_tree = []

In [ ]:
import json
from anytree import Node, RenderTree

# Read the lines from the file
with open(comments_file, 'r') as file:
    lines = file.readlines()

# Create a dictionary to store the comment nodes by their IDs
conversations_tree = []
flat_comments_list = []
# Process each line
for line in lines:
    comment_nodes = {}
    # Parse the JSON object
    json_obj = json.loads(line)
    # Extract the relevant information from the JSON object
    submission_id = json_obj['submission_id']
    comments = json_obj['comments']
    # Create root node for the submission
    ####replace it with the actual body
    root = Node(submission_id)
    comment_nodes[submission_id] = root
    flat_comments_list.append({'id': submission_id, 'body': "", 'author':None, 'reply_to':None, 'conversation_id':submission_id})
    # Create child nodes for the comments
    for comment in comments:
        # Check if comment is from a bot and ignore it if it is
        if "I am a bot, and this action was performed automatically" in comment['body']:
            continue

        comment_id = comment['id']
        author = comment.get('author_fullname', 'unknown')
        parent_id = comment['parent_id'].split('_', 1)[-1]
        body = comment['body']
        # Create a node for the comment with body and ID as attributes
        comment_node = Node(comment_id, body=body)
        flat_comments_list.append({'id': comment_id, 'body': body, 'author':author, 'reply_to':parent_id, 'conversation_id':submission_id})
        if parent_id in comment_nodes:
            # Add the comment node as a child to its parent
            parent_node = comment_nodes[parent_id]
            comment_node.parent = parent_node
        else:
            # Parent ID is equal to submission ID, make comment a child of the root
            comment_node.parent = root
            print("something is wrong")

        # Store the comment node in the dictionary
        comment_nodes[comment_id] = comment_node
    # Print the tree structure
    # print(RenderTree(root))
    conversations_tree.append(root)
#print(RenderTree(conversations_tree[2555]))

In [ ]:
flat_comments_list[:3]

[{'id': 'xsyg0d',
  'body': '',
  'author': None,
  'reply_to': None,
  'conversation_id': 'xsyg0d'},
 {'id': 'iqmwd8f',
  'body': "I've been out looking for Ghoulia today, good to know why I couldn't find her.",
  'author': 't2_57iup',
  'reply_to': 'xsyg0d',
  'conversation_id': 'xsyg0d'},
 {'id': 'iqn4okm',
  'body': 'yeah, fl really got screwed over. was hoping to find my ghouls today but not even a peep, and the workers didn’t even know when they’d be getting stock. i just hope they go online soon',
  'author': 't2_mwcj2qdc',
  'reply_to': 'xsyg0d',
  'conversation_id': 'xsyg0d'}]

In [ ]:
def flatten_tree(node, flattened_tree):
    flattened_tree.append(node)
    for child in node.children:
        flatten_tree(child, flattened_tree)

In [ ]:
flattened_tree = []
flatten_tree(root, flattened_tree)

In [ ]:
# there is node without body!

In [ ]:
from convokit import Corpus, Utterance, Speaker
# Create a Corpus object from the flattened tree.
corpus = Corpus(utterances=[
        Utterance(id=node['id'], text=node['body'], speaker=Speaker(id = node['author']), reply_to=node['reply_to'], conversation_id=node['conversation_id'])
        for node in flat_comments_list
])

In [ ]:
corpus = corpus.fit(corpus)

AttributeError: ignored

In [ ]:
counter = 0
for convo in corpus.iter_conversations():
    counter+=1
print(counter)

2940


In [ ]:
corpus.print_summary_stats()

Number of Speakers: 20381
Number of Utterances: 37647
Number of Conversations: 2940


In [ ]:
from convokit.text_processing import TextParser
parser = TextParser(verbosity=100)
corpus = parser.transform(corpus)

100/37647 utterances processed
200/37647 utterances processed
300/37647 utterances processed
400/37647 utterances processed
500/37647 utterances processed
600/37647 utterances processed
700/37647 utterances processed
800/37647 utterances processed
900/37647 utterances processed
1000/37647 utterances processed
1100/37647 utterances processed
1200/37647 utterances processed
1300/37647 utterances processed
1400/37647 utterances processed
1500/37647 utterances processed
1600/37647 utterances processed
1700/37647 utterances processed
1800/37647 utterances processed
1900/37647 utterances processed
2000/37647 utterances processed
2100/37647 utterances processed
2200/37647 utterances processed
2300/37647 utterances processed
2400/37647 utterances processed
2500/37647 utterances processed
2600/37647 utterances processed
2700/37647 utterances processed
2800/37647 utterances processed
2900/37647 utterances processed
3000/37647 utterances processed
3100/37647 utterances processed
3200/37647 uttera

In [ ]:
from convokit import politenessStrategies
ps = politenessStrategies.PolitenessStrategies(verbose = 1000)
corpus = ps.fit_transform(corpus=corpus)

1000/37647 utterances processed
2000/37647 utterances processed
3000/37647 utterances processed
4000/37647 utterances processed
5000/37647 utterances processed
6000/37647 utterances processed
7000/37647 utterances processed
8000/37647 utterances processed
9000/37647 utterances processed
10000/37647 utterances processed
11000/37647 utterances processed
12000/37647 utterances processed
13000/37647 utterances processed
14000/37647 utterances processed
15000/37647 utterances processed
16000/37647 utterances processed
17000/37647 utterances processed
18000/37647 utterances processed
19000/37647 utterances processed
20000/37647 utterances processed
21000/37647 utterances processed
22000/37647 utterances processed
23000/37647 utterances processed
24000/37647 utterances processed
25000/37647 utterances processed
26000/37647 utterances processed
27000/37647 utterances processed
28000/37647 utterances processed
29000/37647 utterances processed
30000/37647 utterances processed
31000/37647 utteran

In [ ]:
ps.summarize(corpus=corpus)

feature_politeness_==Please==                 0.015220
feature_politeness_==Please_start==           0.028183
feature_politeness_==HASHEDGE==               0.198528
feature_politeness_==Indirect_(btw)==         0.000186
feature_politeness_==Hedges==                 0.084841
feature_politeness_==Factuality==             0.058889
feature_politeness_==Deference==              0.014822
feature_politeness_==Gratitude==              0.041783
feature_politeness_==Apologizing==            0.008739
feature_politeness_==1st_person_pl.==         0.068956
feature_politeness_==1st_person==             0.251999
feature_politeness_==1st_person_start==       0.201291
feature_politeness_==2nd_person==             0.233724
feature_politeness_==2nd_person_start==       0.053789
feature_politeness_==Indirect_(greeting)==    0.009669
feature_politeness_==Direct_question==        0.050416
feature_politeness_==Direct_start==           0.088188
feature_politeness_==HASPOSITIVE==            0.413393
feature_po

In [ ]:
from anytree import Node, find
corpus.get_utterance("iqp0hc8")
target_node = find(conversations_tree, lambda node: node.name == "iqp0hc8")
print(target_node.body)

AttributeError: ignored

In [ ]:
corpus.get_conversation("xsyg0d").meta

ConvoKitMeta({})

In [ ]:
def calculate_conversation_score(dataset_with_metrics_score, metrics_weights):
  i = 2

In [ ]:
file_path = base_path + 'train.tsv.tar.gz'

In [ ]:
import tarfile
import gzip
import pandas as pd

# Specify the path to the .tsv.tar.gz file

# Open the .tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    # Extract the .tsv.gz file from the .tar.gz archive
    tsv_file = tar.getmembers()[0]
    file_extension = tsv_file.name.split('.')[-1]

    # Check if the file is compressed
    if file_extension == 'gz':
        # Open the .tsv.gz file
        with gzip.open(tar.extractfile(tsv_file), 'rt') as tsv:
            # Read the lines and store them in a list
            lines = [line.strip() for line in tsv]

    else:
        # Open the uncompressed .tsv file
        with tar.extractfile(tsv_file) as tsv:
            # Read the lines and store them in a list
            lines = [line.decode().strip() for line in tsv]

# Create a DataFrame from the lines
df = pd.DataFrame([line.split('\t') for line in lines])

# Optionally, you can set the column names based on the first row
df.columns = df.iloc[0]
df = df[1:]  # Remove the first row (column names)

# Print the DataFrame
print(df)


KeyboardInterrupt: ignored

In [ ]:
import tarfile
import gzip
import pandas as pd

# Specify the path to the .tsv.tar.gz file
#file_path = 'path/to/your/file.tsv.tar.gz'

# Specify the number of lines to read at a time
batch_size = 1000

# Open the .tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    # Extract the .tsv.gz file from the .tar.gz archive
    tsv_file = tar.getmembers()[0]
    file_extension = tsv_file.name.split('.')[-1]

    # Check if the file is compressed
    if file_extension == 'gz':
        # Open the .tsv.gz file
        with gzip.open(tar.extractfile(tsv_file), 'rt') as tsv:
            # Read the lines in batches
            lines = []
            for line in tsv:
                lines.append(line.strip())
                if len(lines) == batch_size:
                    # Process the lines and create a DataFrame
                    df = pd.DataFrame([line.split('\t') for line in lines])

                    # Optionally, set the column names based on the first row
                    df.columns = df.iloc[0]
                    df = df[1:]  # Remove the first row (column names)

                    # Do further processing with the DataFrame
                    # ...

                    # Clear the lines list to free memory
                    lines = []

            # Process the remaining lines
            if lines:
                df = pd.DataFrame([line.split('\t') for line in lines])
                df.columns = df.iloc[0]
                df = df[1:]

    else:
        # Open the uncompressed .tsv file
        with tar.extractfile(tsv_file) as tsv:
            # Read the lines in batches
            lines = []
            for line in tsv:
                lines.append(line.decode().strip())
                if len(lines) == batch_size:
                    # Process the lines and create a DataFrame
                    df = pd.DataFrame([line.split('\t') for line in lines])

                    # Optionally, set the column names based on the first row
                    df.columns = df.iloc[0]
                    df = df[1:]  # Remove the first row (column names)

                    # Do further processing with the DataFrame
                    # ...
                    print(df.columns)
                    # Clear the lines list to free memory
                    lines = []

            # Process the remaining lines
            if lines:
                df = pd.DataFrame([line.split('\t') for line in lines])
                df.columns = df.iloc[0]
                df = df[1:]

# Print the DataFrame
#print(df)
